In [1]:
# 6-1
!mkdir chap6
%cd ./chap6

/Users/maton/BERT_Practice/chap6


/Users/maton/.pyenv/versions/3.10.13/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# 6-3
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [4]:
# 6-4（説明コメント付き：日本語BERTによる二値分類のセットアップ）
# =====================================================================
# 目的：
#  - 事前学習済みの日本語BERT（MODEL_NAME）を、Sequence Classification（文分類）タスク用ヘッド付きでロードし、
#    デバイス（MacのMPS / CUDA / CPU）に配置する。
#
# 理論メモ：
#  - BertForSequenceClassification は BERT 本体の上に「分類ヘッド（Dropout→Linear）」を載せたモデルである。
#  - 推論時の出力 logits は形状 [B, num_labels]。num_labels=2 のとき、
#    softmax(logits) によりクラス確率（2クラス）が得られる（argmax で予測ラベル）。
#  - 学習時の損失は入力ラベルと num_labels により自動選択される（HFの既定）：
#      * num_labels == 1 かつラベルが float       → 回帰（MSELoss）
#      * num_labels > 1 かつラベルが整数カテゴリ → 単一ラベル分類（CrossEntropyLoss）
#      * それ以外（ラベルが multi-hot 等）      → マルチラベル（BCEWithLogitsLoss）
#  - BERT の文表現は概ね [CLS] トークンの隠れ状態（最終層）を用い、分類ヘッドで写像する設計（プール層の出力を使う実装もある）。
#  - トークナイザは [CLS]/[SEP]/[PAD] を自動付与（既定）し、attention_mask でPADを無視する。
# =====================================================================

import torch
from transformers import BertJapaneseTokenizer, BertForSequenceClassification


# --- デバイス選択（Mac優先：MPS → CUDA → CPU） ---
def get_best_device() -> torch.device:
    # Apple Silicon + macOS Metal（CUDAは不可）
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    # 他環境（Linux/Windows等）ではCUDAを優先
    if torch.cuda.is_available():
        return torch.device("cuda")
    # それ以外はCPU
    return torch.device("cpu")


device = get_best_device()
print(f"[info] using device = {device}")

# --- モデル名の決定（未定義時の保険） ---
try:
    MODEL_NAME
except NameError:
    # WWM版：語単位での一貫性を意識したマスキングで事前学習（日本語に適した設定）
    MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

# --- トークナイザのロード ---
# 形態素解析→WordPieceの二段分割。語彙はモデルとペアで一致させる（IDずれ防止）。
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# --- 文分類モデル（2クラス）をロード ---
# num_labels=2：二値分類（例：ネガ/ポジ、真/偽 など）
# 出力 logits 形状：[B, 2]。学習時にラベルが整数（0/1）なら CrossEntropyLoss が自動選択される。
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# --- モデルを最適デバイスへ配置 ---
# 元コード：bert_sc = bert_sc.cuda()
# → Mac の場合 .cuda() は不可。統一して .to(device) を用いる。
bert_sc = bert_sc.to(device)

# --- 推論時の基本姿勢 ---
# 評価モードにして Dropout を停止（推論の安定化）。学習時は bert_sc.train() に切り替える。
bert_sc.eval()

# ---------------------------------------------------------------------
# 参考：推論の最小例（コメント解除で動作確認）
# ---------------------------------------------------------------------
# texts = ["今日は自然言語処理を学ぶ。", "この映画は退屈だった。"]
# enc = tokenizer(
#     texts,
#     max_length=128,           # 学習・評価で固定長を揃えると効率的（pad_to_max_length 相当）
#     padding="max_length",
#     truncation=True,
#     return_tensors="pt"
# )
# enc = {k: v.to(device) for k, v in enc.items()}
# with torch.no_grad():
#     logits = bert_sc(**enc).logits           # [B, 2]
#     probs  = logits.softmax(dim=-1)          # クラス確率
#     preds  = probs.argmax(dim=-1)            # 予測クラスID（0/1）
# print("probs:", probs.detach().cpu().numpy())
# print("preds:", preds.detach().cpu().tolist())

# ---------------------------------------------------------------------
# 運用上の注意（理論と実務の橋渡し）
# ---------------------------------------------------------------------
# - 前処理の一貫性：学習・評価で tokenizer の設定（max_length, padding, truncation）が一致していること。
# - ラベル整合性：データセット内で 0/1 とクラス名のマッピングを厳密に固定（モデル保存時に一緒に管理）。
# - 不均衡データ：クラス不均衡が強い場合は、重み付き損失（class_weight）や閾値最適化で改善。
# - 再現性：random seed、モデルバージョン、辞書バージョンを固定。デバイス差（MPS/CUDA/CPU）で微差が出る点は許容範囲で管理。
# - 転移学習：タスクに合わせて最終ヘッドのみならずBERT側も微調整（学習率は本体を小さく、ヘッドを大きくするのが通例）。

[info] using device = mps


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [5]:
# 6-5（説明コメント付き：二値文分類のバッチ推論と精度計算）
# =====================================================================
# 目的：
#  - 日本語BERTの文分類モデル（bert_sc; num_labels=2）を用いて、複数文の推論を一括で行い、
#    予測ラベルと単純精度（accuracy）を計算・表示する。
#
# 理論メモ：
#  - BertForSequenceClassification の出力 logits 形状は [B, num_labels]。
#    num_labels=2 のとき argmax(logits) は「2クラスsoftmaxの最大事後確率クラス」と一致する。
#  - tokenizer(..., padding='longest') はバッチ内の最長系列に合わせてPADを付与する
#    “動的パディング”。attention_mask により PAD 位置は Self-Attention から無視される。
#  - 学習時はラベル（0/1）が整数クラスなら CrossEntropyLoss が自動選択される（HFの既定）。
#    本セルは推論のみ（no_grad & eval）。
#  - デバイス（MPS/CUDA/CPU）は“モデルが現在置かれている device”に合わせて入力テンソルを移動する。
#    （Mac は通常 MPS。`.cuda()` は不可なので `.to(device)` を一貫して用いる。）
# =====================================================================

text_list = [
    "この映画は面白かった。",  # 正例のつもり（1）
    "この映画の最後にはがっかりさせられた。",  # 負例のつもり（0）
    "この映画を見て幸せな気持ちになった。",  # 正例のつもり（1）
]
label_list = [1, 0, 1]  # 教師ラベル（0=negative, 1=positive の想定）

# --- データの符号化（トークン化→ID化） ---
# padding='longest'：バッチ内の最長文の長さに合わせてPADを付与（動的パディング）。
# return_tensors='pt'：PyTorchテンソルで返す（input_ids/attention_mask/token_type_ids など）。
# ＊長文や学習時は max_length と truncation=True を併用して長さ上限を明示するのが安定。
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")

# --- デバイスをモデルに揃える（MPS/CUDA/CPU いずれでも動く書き方） ---
# bert_sc は 6-4 で .to(device) 済みの前提。ここではその device を参照して入力を移す。
device = next(bert_sc.parameters()).device
encoding = {k: v.to(device) for k, v in encoding.items()}

# ラベルテンソル作成
# dtype=torch.long：分類（CE Loss）のクラスIDは long 整数で表現するのが通例。
labels = torch.tensor(label_list, dtype=torch.long, device=device)

# --- 推論（評価モード＋勾配無効） ---
# eval()：Dropout等を停止して推論安定化。6-4で eval 済みでも冪等。
bert_sc.eval()
with torch.no_grad():
    # モデル呼び出し：.forward(**encoding) と bert_sc(**encoding) は等価（後者が慣用的）
    output = bert_sc(**encoding)

# --- 出力の解釈と評価 ---
scores = output.logits  # 形状 [B, 2]：各クラスのスコア（ロジット）
labels_predicted = scores.argmax(-1)  # 予測クラスID（最尤クラス）
num_correct = (labels_predicted == labels).sum().item()  # 正解数（スカラー）
accuracy = num_correct / labels.size(0)  # 単純精度（micro accuracy）

# --- 可視出力（形状と予測・精度） ---
print("# scores (shape):")
print(scores.size())  # 例：torch.Size([3, 2])
print("# predicted labels:")
print(labels_predicted)  # 例：tensor([1, 0, 1], device=..., dtype=torch.int64)
print("# accuracy:")
print(accuracy)  # 例：1.0

# 運用上の注意：
# - データ分布が偏っている場合、単純精度だけでは不十分。適宜、適合率/再現率/F1、混同行列を併用する。
# - 推論の閾値調整が必要な場合（特にコスト非対称なタスク）は、2クラスsoftmaxの確率を取得して閾値最適化を行う。
# - 学習と評価で tokenizer の前処理（max_length/padding/truncation）を一致させること。
# - デバイス差（MPS/CUDA/CPU）でごく僅かな数値差が出る場合があるが、多くは統計誤差の範囲。

# scores (shape):
torch.Size([3, 2])
# predicted labels:
tensor([1, 1, 1], device='mps:0')
# accuracy:
0.6666666666666666


In [6]:
# 6-6（説明コメント付き：ラベル付き入力で損失を計算）
# =====================================================================
# 目的：
#  - 3本のテキストを一括でトークン化し、ラベル（0/1）を入力に同梱して
#    BertForSequenceClassification が内部で計算する損失（loss）を取得して表示する。
#
# 理論メモ：
#  - Hugging Face の BertForSequenceClassification は、forward(**batch) に
#    'labels' を渡すと自動で損失を計算して返す。
#    * num_labels=2 かつ labels が整数クラス（dtype: long）の場合 → CrossEntropyLoss
#    * 出力 logits の形状は [B, num_labels]。loss はバッチ平均のスカラー。
#  - tokenizer(..., padding='longest') は“動的パディング”（バッチ内の最長系列に合わせてPAD）。
#    attention_mask により PAD 位置の注意は抑制される。
#  - デバイス整合：モデル（bert_sc）の置かれている device（MPS/CUDA/CPU）に
#    入力テンソルを合わせること（.to(device)）。Mac では .cuda() は不可。
# =====================================================================

# 符号化（トークン化→ID化）。長さはバッチ内最長に合わせてPAD（学習では max_length+truncation推奨）。
encoding = tokenizer(text_list, padding="longest", return_tensors="pt")

# ラベルを入力 dict に追加。
# - CrossEntropyLoss の前提として dtype は long（整数クラスID）。
# - デバイス移動はこの後まとめて行うので、ここではCPUでOK。
encoding["labels"] = torch.tensor(label_list, dtype=torch.long)

# デバイス整合：モデルの実デバイスへ（Macは多くの場合 'mps'）。
device = next(bert_sc.parameters()).device
encoding = {k: v.to(device) for k, v in encoding.items()}

# ロスの計算
# - labels を渡しているため、出力に loss が含まれる。
# - 学習（逆伝播）するなら bert_sc.train() にして、loss.backward() → optimizer.step() を行う。
#   ここでは損失値の確認のみ。
output = bert_sc(**encoding)
loss = output.loss  # スカラー損失（バッチ平均）

print(loss)

# 補足（運用上の注意）：
# - 動的パディングはバッチごとに系列長が変わるため、学習ループでは DataCollator で揃えるか、
#   max_length + truncation + padding='max_length' で固定長にすると効率が安定する。
# - クラス不均衡が大きい場合、重み付き損失や閾値最適化、評価指標（F1等）の併用が有効。

tensor(0.5852, device='mps:0', grad_fn=<NllLossBackward0>)


In [7]:
# 6-7
#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz 
#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz 

--2025-11-23 10:54:21--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
www.rondhuit.com (www.rondhuit.com) をDNSに問いあわせています... 59.106.19.174
www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 8855190 (8.4M) [application/x-gzip]
`ldcc-20140209.tar.gz' に保存中

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.33MB/s 時間 5.7s       

2025-11-23 10:54:28 (1.49 MB/s) - `ldcc-20140209.tar.gz' へ保存完了 [8855190/8855190]



In [8]:
!cat ./text/it-life-hack/it-life-hack-6342280.txt # ファイルを表示

http://news.livedoor.com/article/detail/6342280/
2012-03-06T13:00:00+0900
USB3.0対応で爆速データ転送！　9倍速のリーダー／ライター登場
USB3.0が登場してから今年で4年目となるがパソコン側でのUSB3.0ポート搭載が進んで来ても対応機器がなかなか充実していない現状がある。そんな中で新しく高速な読み取りが可能なメモリーカードリーダー／ライターが登場した。

バッファローコクヨサプライがUSB3.0対応のカードリーダー／ライターを発表した。SDHC対応のSD系メディアやコンパクトフラッシュ、メモリースティック系メディア、xDピクチャーカードといったデジカメやスマホ、携帯ゲームといった機器で使われている各種メディアを従来よりも短時間でPCに取り込むことが可能になる。

転送速度が5Gbps（理論値）とUSB2.0の480Mbpsと比べて爆速になったUSB3.0はPC側の対応が進んで来ていたが高速転送が生かせる周辺機器としては、外付けHDDや一部のUSBメモリーくらいしかなかった。これに多くのメディアが扱えるリーダー／ライターが加わることで手軽にUSB3.0の恩恵を受けることができるようになる。

今回発表されたのは、USB3.0ケーブルとカードリーダー本体が分かれるタイプの「BSCR09U3」シリーズ（3,240円）、USB3.0コネクタをカードリーダー本体に内蔵している「BSCRD04U3」シリーズ（2,690円）だ。共にホワイトとブラックのカラーバリエーションが用意される（発売は3月下旬以降）。

■リリースページ
■バッファローコクヨサプライ




■バッファローの記事をもっと見る
・約283gでカバンに入る！小型キーボードの驚くべき機能
・3種類のホットキーで使いやすい！AndroidとPCで使えるキーボードの魅力
・ドラえもんもビックリの新アイテム！マウスとキーボードが合体"OPAir"
・ありそうでなかった便利機能！ファイル仕分けする画期的なHDD


サンディスク SanDisk microSDHC 32GB（microSD 32GB） 超高速クラス4  変換アダプター付 世界国内シェアNo.1 バルク品
クチコミを見る


In [9]:
# 6-9（説明コメント付き：PyTorch DataLoader でミニバッチを取り出す最小例）
# ======================================================================
# 目的：
#  - list[dict] で用意した玩具データから DataLoader を構築し、ミニバッチ単位で取り出す流れを確認する。
#
# 理論メモ：
#  - DataLoader は「確率的勾配降下（SGD）」系学習における **ミニバッチ化** を担う。
#    統計的には、全データの勾配（バッチ平均）を近似しつつ、適度なノイズで汎化を促進する。
#  - collate（結束）規則：デフォルト collate_fn は **辞書の各キーを軸にテンソル結合** する。
#    つまり list[{"data": t1, "labels": y1}, {"data": t2, "labels": y2}] →
#    {"data": stack([t1, t2]), "labels": stack([y1, y2])}
#  - 分類タスクのラベルは通常 **整数（torch.long）**。CrossEntropyLoss は long クラスIDを想定する。
#  - 実運用では DataLoader のパラメータ（shuffle, num_workers, pin_memory, drop_last 等）が
#    収束性やスループット、再現性に影響する（後述の注意参照）。
# ======================================================================

# データローダーの作成
# - 各要素は dict で、"data" に特徴ベクトル、"labels" にクラスID（スカラー）を持つ。
# - ここでは玩具データとして 2次元ベクトル＋ラベルを定義（ラベル0～3 → 4クラス相当）。
#   ※ 直前までの BERT 例（num_labels=2）とは設定が異なる点に注意（あくまで DataLoader の例）。
dataset_for_loader = [
    {"data": torch.tensor([0, 1]), "labels": torch.tensor(0)},  # 例：サンプル1
    {"data": torch.tensor([2, 3]), "labels": torch.tensor(1)},  # 例：サンプル2
    {"data": torch.tensor([4, 5]), "labels": torch.tensor(2)},  # 例：サンプル3
    {"data": torch.tensor([6, 7]), "labels": torch.tensor(3)},  # 例：サンプル4
]
# - batch_size=2：2件ずつ結束。shuffle を付ければ各エポックで順番をランダム化できる。
# - デフォルト collate_fn により、辞書キーごとにテンソルが stack される。
loader = DataLoader(dataset_for_loader, batch_size=2)

# データセットからミニバッチを取り出す
# - enumerate(loader) は各反復で dict({"data": Tensor[B, ...], "labels": Tensor[B]}) を返す。
# - 実際のファインチューニングでは、このループ内で
#   1) device へ転送 → 2) forward → 3) loss 計算 → 4) backward → 5) optimizer.step → 6) zero_grad
#   を行う（評価時は no_grad + eval()）。
for idx, batch in enumerate(loader):
    print(f"# batch {idx}")
    print(batch)
    ## ファインチューニングではここでミニバッチ毎の処理を行う
    # 例（擬似手順；実コードはモデルと損失に依存）：
    # - batch = { "data": Tensor[B, D], "labels": Tensor[B] }
    # - features = batch["data"].to(device)         # モデルの device（CPU/MPS/CUDA）へ
    # - targets  = batch["labels"].to(device).long()# CE を使うなら long へ
    # - logits   = model(features)                  # 前向き計算
    # - loss     = criterion(logits, targets)       # 例：CrossEntropyLoss
    # - loss.backward(); optimizer.step(); optimizer.zero_grad()

# ----------------------------------------------------------------------
# 実務上の注意（BERT/Transformers 文脈へのブリッジ）
# ----------------------------------------------------------------------
# - 本例は玩具の "data"/"labels" 構造だが、Transformers では通常
#   {"input_ids", "attention_mask", "token_type_ids", "labels"} の dict を DataLoader が返す。
# - トークン列は長さがまちまちのため、DataCollator（例：DataCollatorWithPadding）で
#   バッチごとに動的パディングするのが一般的。固定長にしたい場合は
#   tokenizer(..., max_length, truncation=True, padding="max_length")。
# - 速度最適化：
#   * num_workers>0 で入出力を並列化（ただし MPS は制約があるため環境依存で調整）。
#   * pin_memory=True（CUDA で有効）、prefetch_factor の調整。
# - 再現性：
#   * 乱数seed固定、shuffle=True の際は Sampler も含めて管理。
#   * drop_last=True はバッチ正規化などで端数バッチを避けたい時に有用。

# batch 0
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}
# batch 1
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}


In [10]:
# 6-10（説明コメント付き：DataLoader をシャッフルしてミニバッチ列挙）
# ======================================================================
# 目的：
#  - 直前セル（6-9）で用意した list[dict] 形式のデータを DataLoader に渡し、
#    shuffle=True で「各エポックごとにサンプル順をランダム化」したうえで、
#    ミニバッチ単位で取り出す挙動を確認する。
#
# 理論メモ：
#  - SGD 系学習では、データ順をランダム化（シャッフル）することで
#    勾配推定のバイアスを減らし、汎化性能を高めやすい（局所最適・順序依存の緩和）。
#  - PyTorch の DataLoader は「エポックの開始時にランダムな順列をサンプル集合に適用」し、
#    その順に batch_size ごとに切り出す（drop_last=False なら端数も返す）。
#  - デフォルトの collate_fn は「同じキーでテンソルを stack」するため、
#    list[{"data": t_i, "labels": y_i}] → {"data": stack(t_i), "labels": stack(y_i)} になる。
#  - 再現性が必要なら、DataLoader(generator=...) に seed を渡すか、
#    グローバルに torch.manual_seed(...) と Sampler を適切に設定する。
# ======================================================================

# DataLoader の生成
# - dataset_for_loader（6-9で定義済み）からバッチサイズ2で取り出す。
# - shuffle=True によって、反復のたびに（厳密にはエポックのたびに）順序が変わり得る。
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)

# ミニバッチの列挙
# - 各反復で返る `batch` は dict で、{"data": Tensor[B, ...], "labels": Tensor[B]} の形。
# - 実際の学習では、このループ内で device 転送 → forward → loss → backward → step を行う。
for idx, batch in enumerate(loader):
    print(f"# batch {idx}")
    print(batch)

    ## ファインチューニングではここでミニバッチ毎の処理を行う
    # 例（擬似手順）：
    # features = batch["data"].to(device)
    # targets  = batch["labels"].to(device).long()  # CE を使うなら long
    # logits   = model(features)
    # loss     = criterion(logits, targets)
    # loss.backward(); optimizer.step(); optimizer.zero_grad()

# 補足：
# - 同じコードを何度実行しても、shuffle=True のためバッチ内容の順序は実行のたびに変わる。
# - 再現性を担保したい場合（ベンチマークや回帰テスト）は、
#   gen = torch.Generator().manual_seed(42)
#   loader = DataLoader(..., shuffle=True, generator=gen)
#   のように「生成器を明示」するのが確実。

# batch 0
{'data': tensor([[2, 3],
        [4, 5]]), 'labels': tensor([1, 2])}
# batch 1
{'data': tensor([[0, 1],
        [6, 7]]), 'labels': tensor([0, 3])}


In [11]:
# 6-11（説明コメント付き：livedoorニュース等のカテゴリ別テキストをBERT学習用に前処理）
# ======================================================================
# 目的：
#  - カテゴリ名のリスト（例：livedoor ニュースコーパス相当）から各カテゴリの記事ファイルを走査し，
#    トークナイザで ID 列へ符号化して，学習用データ（辞書の配列）を構築する。
#
# 理論メモ：
#  - 分類タスクでは，各サンプルは {input_ids, token_type_ids, attention_mask, labels} の辞書で表すのが標準。
#  - BERT 系は [CLS] を用いて文（文書）表現を得るため，長文は max_length で**切り詰め（truncation）**される。
#    → 重要情報が後半にある記事は情報落ちのリスク。必要に応じ「スライディングウィンドウ」や要約等を検討。
#  - padding='max_length' により**固定長化**することでミニバッチの効率・再現性が上がる（メモリ見積りが安定）。
#  - labels は整数クラス ID（long 型）で CrossEntropyLoss に適合する。カテゴリ順序＝ラベル割当になるため，
#    後段の評価・可視化で**クラス名との対応表**を必ず保存すること。
# ======================================================================

# 前提（このセルの外でインポート済みであること）：
#   from tqdm import tqdm
#   import glob
#   import torch
#   from transformers import BertJapaneseTokenizer
#   MODEL_NAME は 6-4 と同一（トークナイザとモデルの語彙は一致させる）

# カテゴリーのリスト
# - enumerate により 0,1,2,... の整数がラベルとして付与される。
# - 順序はそのままクラスIDになるため、将来の再現・可視化のためにこのリストを保存しておくこと。
category_list = [
    "dokujo-tsushin",
    "it-life-hack",
    "kaden-channel",
    "livedoor-homme",
    "movie-enter",
    "peachy",
    "smax",
    "sports-watch",
    "topic-news",
]

# トークナイザのロード
# - BERT 本体と同じ MODEL_NAME を使用（語彙IDの不一致を防ぐ）。
# - BertJapaneseTokenizer は形態素解析→WordPiece の二段分割を内部で行う。
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 各データの形式を整える
# - max_length：系列長の上限。大きくすると情報保持は増えるが，計算負荷とメモリも増える。
# - 固定長（padding='max_length'）はミニバッチ効率を安定させる。
max_length = 128
dataset_for_loader = []

# ループ構造：
# - カテゴリごとにファイルを探索し（glob），各ファイルから本文を抽出→トークナイズ→テンソル化→labels 付与。
# - tqdm により進捗が可視化される（長時間処理の観測性を確保）。
for label, category in enumerate(tqdm(category_list)):
    # ファイル探索：
    # - 例：./text/it-life-hack/it-life-hack* のようなパターンをすべて拾う。
    # - ディレクトリ構成はデータセット依存。I/O 例外やエンコーディングの揺れがある場合は try/except と encoding 指定を検討。
    for file in glob.glob(f"./text/{category}/{category}*"):
        # 記事の読み取り：
        # - livedoor コーパス慣例では 1行目:URL, 2行目:日付, 3行目:タイトル, 4行目以降:本文 となっているため，
        #   ここでは 4 行目（index=3）以降を本文として使用している。
        # - 改行は '\n' で連結。open の文字コードは環境により 'utf-8' 明示が安全。
        lines = open(file).read().splitlines()
        text = "\n".join(
            lines[3:]
        )  # ファイルの4行目からを抜き出す（タイトル・メタ情報は除外）。

        # トークナイズ：
        # - max_length：上限に達したトークンは切り詰め（truncation=True）。
        # - padding='max_length'：不足分は [PAD] で埋め，attention_mask=0 で無視される。
        # - 戻り値は dict（input_ids/token_type_ids/attention_mask などのリスト）。
        encoding = tokenizer(
            text, max_length=max_length, padding="max_length", truncation=True
        )

        # ラベル付与：
        # - enumerate の整数（0..）をそのままクラスIDに採用。
        # - CrossEntropyLoss を想定するなら long 整数であることが前提（この後の tensor 化で既定は int64=long）。
        encoding["labels"] = label

        # テンソル化：
        # - DataLoader で stack できるよう，各フィールドを torch.tensor に変換。
        # - dtype は自動推論されるが，必要に応じて labels を明示的に dtype=torch.long にしても良い。
        encoding = {k: torch.tensor(v) for k, v in encoding.items()}

        # データ配列へ格納：
        # - 1 サンプル = 1 dict。後段で DataLoader に渡すとキーごとに stack されてミニバッチになる。
        dataset_for_loader.append(encoding)

# 補足（運用上の注意）：
# - データ分割：本コードは全件を単一リストに溜める。**学習/検証/テストの分割**は別途必須（ランダム分割や時間分割等）。
# - クラス不均衡：カテゴリ間でサンプル数が偏ると学習が偏る。重み付き損失やリサンプリングを検討。
# - 文字コード：open(file, encoding='utf-8') などの明示が環境により必要。I/O エラー処理も実務では追加。
# - 長文対策：128 トークンで情報が欠落する場合は，max_length を増やす／スライディングウィンドウで分割／要約前処理を検討。
# - 再現性：MODEL_NAME，トークナイザ・辞書バージョン，カテゴリ順序→ラベル対応をメタデータとして保存する。

100%|██████████| 9/9 [00:16<00:00,  1.78s/it]


In [12]:
# 6-12
print(dataset_for_loader[0])

{'input_ids': tensor([    2,   227, 28553,   227,   687,  4847,    80,     8,   858,    19,
           40,  6309, 10598,    11,   654,  1174,    15,    16,    33,     5,
            9, 21411, 15933,    14,  1460,    34, 18447,  9699, 18447,  9541,
          725,  9878, 28511,     8,  4726,     7,  2575,  3290,  1624, 31041,
         2612,    11,  2949,  5835,    16, 19199,    13,   969,   558,    40,
         5847,    81,    18,  1913,    11,   489,    16,    33,     8, 15933,
          811,   737,    12,     9,     5,  3892,     7, 24169,    15,    10,
        10494,    13,    59,    73,  3062,     9,  2935,  1766,  8881, 28756,
          235,    40,  8962,  1978,   155,     8,    73, 29928,    11,   454,
        28470,   191,    16,  1096, 28555,  5370,     5, 18447,  9699, 18447,
         9541,   725,  9878, 28511,    11,  3111,    10,     8, 20964,     9,
          259, 29064,    13,   625,  3005,    11,  4748, 31260,    34,  1101,
        14974,    16,    73, 28363,    75,     8, 

In [13]:
# 6-13
# =========================================================
# 説明のコメント付き：学習/検証/テスト分割と DataLoader 構築
# ---------------------------------------------------------
# 目的：
#  - 事前に前処理済み（dict: input_ids/attention_mask/token_type_ids/labels …）の
#    dataset_for_loader を 60/20/20 に分割し、学習・検証・テスト用の DataLoader を作成する。
#
# 理論メモ：
#  - ランダム分割は単純だが、**クラス不均衡**や**時系列依存**がある場合には性能推定が歪む。
#    → 分類では本来 **層化分割（stratified split）** が望ましい（各クラス比率を保つ）。
#  - 検証（val）はハイパーパラメータ・早期終了判定に使用、テスト（test）は最終一度だけ触れるのが原則。
#  - DataLoader はミニバッチ化の責務：学習のみ shuffle=True（勾配推定のバイアス低減）。
#    検証・テストは順序固定（再現性/デバッグ容易化）。
#  - 再現性：random.shuffle は**シード未固定**だと毎回分割が変わる。必要なら外部で seed 固定。
# =========================================================

# データセットの分割
random.shuffle(
    dataset_for_loader
)  # ランダムにシャッフル（※再現性が必要なら事前に seed 固定）
n = len(dataset_for_loader)  # 全サンプル数
n_train = int(0.6 * n)  # 60% を学習に
n_val = int(0.2 * n)  # 20% を検証に
# 残り（約20%）はテストへ（端数は test に回る）
dataset_train = dataset_for_loader[:n_train]  # 学習データ
dataset_val = dataset_for_loader[n_train : n_train + n_val]  # 検証データ
dataset_test = dataset_for_loader[n_train + n_val :]  # テストデータ
# ＊注意：分類タスクでクラス比率を保ちたい場合は、このランダム分割ではなく層化分割を用いる。
#   （例：各クラスごとに同割合で train/val/test に落とし込む）

# データセットからデータローダを作成
# - DataLoader は dict のキーごとにテンソルを stack してミニバッチを作る（デフォルト collate）。
# - 学習データは shuffle=True（各エポックで順序を無作為化して汎化性能向上を狙う）。
# - バッチサイズ設計：
#   * train は 32（学習安定と計算資源のバランス）。VRAM とスループットに応じて調整。
#   * val/test は大きめ（256）で高速化（勾配計算なし前提）。VRAM に収まらない場合は小さく。
# - drop_last：端数バッチを落としたい（BN等）場合に True を検討。BERT 文分類では通常 False で可。
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_val = DataLoader(
    dataset_val, batch_size=256
)  # 検証は順序固定（shuffle=False 既定）
dataloader_test = DataLoader(dataset_test, batch_size=256)  # テストも順序固定

# 補足：
# - 乱数一貫性：エポック間・実行間の再現性が必要なら、分割前に random.seed(...) を設定。
#   DataLoader のシャッフル側は generator=... で seed を渡す方法もある。
# - 大規模データ：num_workers, pin_memory（CUDAのみ）, persistent_workers などの I/O 最適化は学習時に検討。
# - ラベル整合：クラス名 ↔ ラベルID（category_list の順序）対応表を必ず保存しておく（評価・可視化に必須）。

In [14]:
# 6-14
# =====================================================================
# 説明のコメント付き：PyTorch Lightning を用いた BERT 文分類の学習ループ定義
# ---------------------------------------------------------------------
# 目的：
#  - Transformers の BertForSequenceClassification を LightningModule に包み、
#    学習・検証・テスト各フェーズの step を最小構成で定義する。
#
# 理論メモ（重要ポイント）：
#  - BertForSequenceClassification は BERT 本体 + Dropout + Linear の「分類ヘッド」を持つ。
#    forward(**batch) に labels（整数クラスID）を渡すと CrossEntropyLoss を**自動計算**して返す。
#  - Lightning は training_step が返した loss を backward → optimizer.step する（標準挙動）。
#    self.log(...) はメトリクスをロガー（TensorBoard等）に出し、on_step/on_epoch の集計方針を指定可能。
#  - 本実装の optimizer は Adam。Transformer では通常 AdamW + weight decay + 学習率スケジューラ
#    （linear warmup/decay）が一般的（改良余地としてコメントに示す）。
#  - test_step の accuracy は「バッチ内平均」であり、エポック平均を厳密に取りたい場合は
#    on_epoch=True で self.log するか、torchmetrics を使って集約するのが推奨。
# =====================================================================


class BertForSequenceClassification_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前（例：'tohoku-nlp/bert-base-japanese-whole-word-masking'）
        # num_labels: ラベルの数（多クラス分類ならクラス数、二値なら 2）
        # lr        : 学習率（BERT 微調整の典型レンジは 2e-5〜5e-5 程度。層別LRやヘッド強めも有効）
        super().__init__()

        # save_hyperparameters():
        # - __init__ の引数（ここでは model_name, num_labels, lr）を self.hparams に保存し、
        #   チェックポイントにも自動で埋め込む（再現性・追跡性のために有用）。
        # - 以後 self.hparams.lr などで参照できる。
        self.save_hyperparameters()

        # BERTのロード：
        # - 事前学習済み BERT + 分類ヘッド（Dropout→Linear）が読み込まれる。
        # - num_labels により最終 Linear 出力次元が [hidden_size → num_labels] になる。
        # - 学習時、labels（long 整数）を渡すと CrossEntropyLoss が内部で適用される（平均リダクション）。
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name, num_labels=num_labels
        )
        # 参考（改善の余地・理論メモ）：
        # - 転移学習の安定化：最初は分類ヘッドのみ学習→徐々にBERT本体を解凍（層別LR）
        # - 正則化：weight decay（AdamW）や勾配クリップ、ドロップアウト率の調整が一般的
        # - スケジューラ：線形ウォームアップ→線形減衰は Transformer 微調整のデファクト

    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数。
    # Lightning は戻り値の loss に対して backward を実行する（標準設定時）。
    def training_step(self, batch, batch_idx):
        # batch は DataLoader からの dict（input_ids, attention_mask, token_type_ids, labels など）
        # labels が含まれていれば、bert_sc(**batch) は loss（CE）を内部計算して返す。
        output = self.bert_sc(**batch)
        loss = output.loss
        # ロギング：
        # - 'train_loss' をログ。必要に応じて on_step=True/on_epoch=True, prog_bar=True 等を指定可能。
        # - 例：self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)
        self.log("train_loss", loss)
        return loss

    # 検証データのミニバッチが与えられた時の評価指標を計算する関数。
    # ここでは損失（CE）のみをログしている。精度やF1が必要なら logits から算出して self.log する。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        # バリデーション損失をログ。エポック平均を取りたい場合は on_epoch=True を明示するのが推奨。
        self.log(
            "val_loss", val_loss
        )  # 例：self.log('val_loss', val_loss, on_step=False, on_epoch=True, prog_bar=True)

    # テストデータのミニバッチが与えられた時の評価指標を計算する関数。
    # ここでは accuracy を「バッチごと」に計算・ログしている（エポック平均の厳密な集計は別設定が必要）。
    def test_step(self, batch, batch_idx):
        # 注意：batch.pop('labels') は batch を破壊的に変更する。再利用があり得る場合はコピー推奨。
        labels = batch.pop("labels")  # バッチからラベルを取り出し、モデル入力から除外
        output = self.bert_sc(**batch)
        # 予測クラス：logits の argmax。softmax 不要（順位のみ使うため）。
        labels_predicted = output.logits.argmax(-1)
        num_correct = (labels_predicted == labels).sum().item()
        accuracy = num_correct / labels.size(0)  # バッチ内精度
        # ロギング：エポック平均を取りたい場合は on_epoch=True を推奨（分散学習では sync_dist=True も）。
        self.log(
            "accuracy", accuracy
        )  # 例：self.log('test_acc', accuracy, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)

    # 学習に用いるオプティマイザを返す関数。
    # 現状は Adam（weight decay 無し）。Transformer の慣例では AdamW + weight_decay + スケジューラが一般的。
    def configure_optimizers(self):
        # 改良案（コメント）：
        # - AdamW に変更：torch.optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=0.01)
        # - スケジューラ：linear warmup (数百〜数千ステップ) → linear decay
        # - Lightning では {"optimizer": opt, "lr_scheduler": sched, "monitor": "val_loss"} の dict を返せる
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [17]:
# 6-15
# =====================================================================
# 説明のコメント付き：学習中のチェックポイント保存と Trainer 設定（PL 2.x 準拠）
# ---------------------------------------------------------------------
# 目的：
#  - 検証指標（val_loss）を監視し、最良モデルの重みを保存する。
#  - 旧API（gpus=...）を廃し、accelerator/devices でデバイスを指定（2.x推奨）。
#
# 理論メモ：
#  - ModelCheckpoint は monitor と mode に従い、より良いモデルのみ保存（save_top_k=1）。
#  - save_weights_only=True は版差で挙動が異なることがあるため、必要に応じ state_dict の別保存も検討。
#  - Lightning 2.x ではデバイス指定は accelerator ("cpu"/"gpu"/"mps"/"auto") と devices（数 or リスト）。
#  - Mac(Apple Silicon) は MPS、NVIDIA 環境は GPU、その他は CPU を使うのが自然。
# =====================================================================

import torch
import pytorch_lightning as pl

# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor="val_loss",  # 監視対象メトリクス（validation_step で self.log される前提）
    mode="min",  # 最小化が良い（損失）
    save_top_k=1,  # ベスト1件のみ保持
    save_weights_only=True,  # 版差に注意（必要なら別途 state_dict も保存）
    dirpath="model/",  # 出力先ディレクトリ
    filename="epoch={epoch}-val_loss={val_loss:.4f}",  # 保存ファイル名に指標を埋め込む
)


# デバイス自動選択（PL 2.x 推奨の指定方法）
def pick_accelerator_and_devices():
    # Apple Silicon (Metal Performance Shaders)
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return "mps", 1
    # CUDA GPU
    if torch.cuda.is_available():
        return "gpu", 1
    # フォールバック：CPU
    return "cpu", 1


accelerator, devices = pick_accelerator_and_devices()

# （任意）早期終了で過学習を抑制：val_loss が改善しなければ打ち切る
early_stopping = pl.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=3)

# 学習の方法を指定（PL 2.x 準拠）
# - max_epochs=10：総エポック数（EarlyStopping と併用で実質の学習長を自動調整）
# - callbacks：チェックポイントと早期終了を適用
trainer = pl.Trainer(
    accelerator=accelerator,  # "mps"（Mac）/ "gpu"（NVIDIA）/ "cpu"
    devices=devices,  # 1（単一デバイス）
    max_epochs=10,
    callbacks=[checkpoint, early_stopping],
    # （必要に応じて）deterministic=True, gradient_clip_val=1.0, precision=16 等を追加
)

# 参考：学習呼び出し例（model は LightningModule、dataloader_* は 6-13 で作成）
# trainer.fit(model, dataloader_train, dataloader_val)

# 参考：学習後のロード
# 最良 ckpt を LightningModule としてロード
# model = BertForSequenceClassification_pl.load_from_checkpoint("model/epoch=..-val_loss=....ckpt")
# あるいは重みのみを別途保存・復元（版差対策）
# torch.save(model.bert_sc.state_dict(), "best_weights.pt")
# model.bert_sc.load_state_dict(torch.load("best_weights.pt", map_location="cpu"))

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


In [ ]:
# 6-16
# =====================================================================
# 説明のコメント付き：LightningModule のインスタンス化と学習実行
# ---------------------------------------------------------------------
# 目的：
#  - 6-14 で定義した LightningModule（BERT 文分類）を生成し，
#    6-13 の DataLoader（train/val）で学習を走らせる。
#
# 理論メモ：
#  - num_labels=9 は 6-11 の category_list の長さに一致（多クラス分類）。
#  - lr=1e-5 は BERT 微調整としては保守的（2e-5〜5e-5 も一般的）。学習が進まない場合は微調整。
#  - Lightning は Trainer 側の accelerator/devices 設定に基づき，テンソルを自動でデバイス転送する。
#  - 6-15 の ModelCheckpoint / EarlyStopping を callbacks に渡していれば，
#    val_loss 最小の ckpt が model/ に保存される（filename に val_loss を埋め込む設定にしておくと追跡が容易）。
# =====================================================================

# PyTorch Lightningモデルのロード（BERT + 分類ヘッド）
# - MODEL_NAME：トークナイザと同一モデル名を使用（語彙IDの不一致を防ぐ）。
# - num_labels=9：9カテゴリ分類（category_list と整合）。
# - lr=1e-5：安定寄り。過学習/収束速度を見て調整（ヘッド大きめ，BERT小さめの層別LRも有効）。
model = BertForSequenceClassification_pl(MODEL_NAME, num_labels=9, lr=1e-5)

# ファインチューニングを実行
# - trainer は 6-15 で PL 2.x 準拠（accelerator/devices）で構築済みの前提。
# - fit() は（学習）→（検証）を各エポックで実行。validation_step の self.log('val_loss', ...) が
#   ModelCheckpoint/EarlyStopping の monitor と一致していることが前提。
trainer.fit(model, dataloader_train, dataloader_val)

# （任意・推奨）学習後のベストモデルでテスト評価を実行する場合：
# - 6-15 で EarlyStopping/Checkpoint を設定済みなら，ckpt_path="best" で自動的に最良重みをロードして評価できる。
# - dataloader_test は 6-13 で構築済みの前提。
# trainer.test(model, dataloader_test, ckpt_path="best")

# （任意）学習後に重みを別途保存したい場合（バージョン差異対策）：
# torch.save(model.bert_sc.state_dict(), "best_weights.pt")
# 復元：model.bert_sc.load_state_dict(torch.load("best_weights.pt", map_location="cpu"))

# 運用上の注意：
# - 早期終了を入れている場合，max_epochs は上限としての意味（実際のエポック数は短くなる）。
# - 指標は accuracy だけでなく macro F1 なども追加すると多クラスでの挙動を把握しやすい。
# - 再現性が必要なら，分割 seed（6-13），Trainer(deterministic=True)，各種乱数 seed の固定を徹底。

In [ ]:
# 6-17
best_model_path = checkpoint.best_model_path  # ベストモデルのファイル
print("ベストモデルのファイル: ", checkpoint.best_model_path)
print("ベストモデルの検証データに対する損失: ", checkpoint.best_model_score)

In [20]:
# 6-18
%load_ext tensorboard
%tensorboard --logdir ./

In [ ]:
# 6-19
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

In [22]:
# 6-20
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(best_model_path)

# Transformers対応のモデルを./model_transformesに保存
model.bert_sc.save_pretrained("./model_transformers")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# 6-21
bert_sc = BertForSequenceClassification.from_pretrained("./model_transformers")